### Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import sys
import os
os.environ["COLAB_GPU"] = "AAA"
import google_colab_selenium as gs
# from concurrent.futures import ThreadPoolExecutor
# import threading
from selenium.webdriver.common.by import By
import time

### Driver selenium

In [2]:
def create_webdriver(url):
    opt = Options()
    #opt.add_argument("--verbose")
    #opt.add_argument('--no-sandbox')
    #opt.add_argument('--headless')
    # opt.add_argument('--disable-gpu')
    # opt.add_argument("--window-size=1920, 1200")
    opt.add_argument('--disable-dev-shm-usage')
    opt.add_argument("--lang=vi-VN")
    opt.add_argument(f"--window-size=1366,768")
    opt.add_argument(f'--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36')
    opt.add_argument('--disable-blink-features=AutomationControlled')
    opt.add_argument("--disable-extensions")
    #opt.add_argument("--proxy-server='direct://'")
    opt.add_argument("--proxy-bypass-list=*")
    opt.add_argument('--ignore-certificate-errors')
    opt.add_argument("--password-store=basic")
    opt.add_argument("--no-sandbox")
    opt.add_argument("--disable-dev-shm-usage")
    opt.add_argument("--disable-extensions")
    opt.add_argument("--enable-automation")
    opt.add_argument("--disable-browser-side-navigation")
    opt.add_argument("--disable-web-security")
    opt.add_argument("--disable-dev-shm-usage")
    opt.add_argument("--disable-infobars")
    opt.add_argument("--disable-gpu")
    opt.add_argument("--disable-setuid-sandbox")
    opt.add_argument("--disable-software-rasterizer")
    #proxy = '171.226.29.241:8080'  # Replace '8080' with the correct port if different
    #opt.add_argument(f'--proxy-server=http://{proxy}')
    driver = gs.Chrome(options=opt)
    time.sleep(5)
    driver.get(url)
    return driver

In [3]:
driver = create_webdriver("https://ads.tiktok.com/business/creativecenter/inspiration/popular/hashtag/pc/vi")

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:3 https://download.docker.com/linux/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Reading package lists...
Building dependency tree...
Reading state information...
1 package can be upgraded. Run 'apt list --upgradable' to see it.
Reading package lists...


Building dependency tree...
Reading state information...
Calculating upgrade...
The following packages will be upgraded:
  tzdata
1 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Need to get 301 kB of archives.
After this operation, 1024 B disk space will be freed.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 tzdata all 2024a-0ubuntu0.20.04.1 [301 kB]


debconf: delaying package configuration, since apt-utils is not installed


Fetched 301 kB in 1s (478 kB/s)
(Reading database ... 66656 files and directories currently installed.)
Preparing to unpack .../tzdata_2024a-0ubuntu0.20.04.1_all.deb ...
Unpacking tzdata (2024a-0ubuntu0.20.04.1) over (2024a-0ubuntu0.20.04) ...
Setting up tzdata (2024a-0ubuntu0.20.04.1) ...
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76.)
debconf: falling back to frontend: Readline

Current default time zone: 'Etc/UTC'
Local time is now:      Fri Jul  5 05:21:09 UTC 2024.
Universal Time is now:  Fri Jul  5 05:21:09 UTC 2024.
Run 'dpkg-reconfigure tzdata' if you wish to change it.



<IPython.core.display.Javascript object>

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  103M  100  103M    0     0  95.3M      0  0:00:01  0:00:01 --:--:-- 95.3M




Reading package lists...
Building dependency tree...
Reading state information...
google-chrome-stable is already the newest version (126.0.6478.126-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
# time.sleep(5)
# cards = driver.find_elements(By.CSS_SELECTOR, "a.CardPc_container___oNb0")
wait = WebDriverWait(driver, 20)
cards = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.CardPc_container___oNb0")))
for card in cards:
    rank = card.find_element(By.CSS_SELECTOR, "span.RankingStatus_rankingIndex__ZMDrH").text.strip()
    hashtag = card.find_element(By.CSS_SELECTOR, "span.CardPc_titleText__RYOWo").text.strip()
    
    posts_views = card.find_elements(By.CSS_SELECTOR, "span.CardPc_itemValue__XGDmG")
    posts = posts_views[0].text.strip()
    views = posts_views[1].text.strip()
    
    print(rank, hashtag, posts, views)
    #print(rank, hashtag)

1 # aifilter 36K 76M
2 # albumcover 35K 43M
3 # capadealbum 33K 31M


In [ ]:
wait = WebDriverWait(driver, 10)
view_more_button = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.ViewMoreBtn_viewMoreBtn__fOkv2[data-testid='cc_contentArea_viewmore_btn']")))
actions = ActionChains(driver)
actions.move_to_element(view_more_button).perform()
view_more_button.click()

In [ ]:
hashtag_cleaned = hashtag.replace("#", "").strip()
hashtag_cleaned

### Create producer send meta-videos (by hashtags) to Kafka

In [4]:
from kafka_producer import send_video_to_kafka, create_producer
import subprocess

def run_test_kafka(hashtag, topic_name):
    try:
        process = subprocess.Popen(["python", "/teamspace/studios/this_studio/KidGuardScan/run_kafka_producer.py", hashtag, topic_name], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        print(stdout)
        if process.returncode != 0:
            print("Error occurred:", stderr)
    except Exception as e:
        print("An error occurred:", str(e))
    finally:
        process.terminate()

In [5]:
run_test_kafka("khoahocdulieu", "Test_with_MongoDB11")